In [1]:
import pandas as pd
import numpy as np
import math
from math import sin, cos, sqrt, atan2
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import warnings
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
plt.style.use('seaborn')

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [3]:
final_scaler = pickle.load(open("scaler.pickle", 'rb'))
final_model =pickle.load(open("model.pickle", 'rb'))

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [4]:
#modifying Date fo make it more readable 
holdout['year'] = holdout['date'].apply(lambda x: int(x[0:4]))
holdout['month'] = holdout['date'].apply(lambda x: int(x[4:6]))

holdout['date'] = holdout['date'].apply(lambda x: x.strip('T000000'))
holdout['date'] = pd.to_datetime(holdout['date'], format='%Y%m%d')

#fixing outliers on bedrooms
holdout[holdout.bedrooms>7].bedrooms=holdout.bedrooms.median()
holdout[holdout.bathrooms>5].bathrooms=holdout.bathrooms.median()

In [5]:
holdout['coordinates'] = list(zip(holdout.lat, holdout.long))
holdout=holdout.drop(['lat','long'],axis=1)

In [6]:
#generated a feature that calculates the distance from city center to add more context to our location columns.
from geopy.distance import geodesic
seattle = (47.60646230355264, -122.33451886696908)
holdout['dis_from_city']=holdout['coordinates'].apply(lambda x: round(geodesic(seattle, x ).miles,2))
holdout.drop(["coordinates"],axis=1,inplace=True)

In [7]:
#binned grade to organize the rating of a house and separated into dummies in reference of how high it was scored.
holdout.grade=pd.cut(holdout.grade,bins=3,labels=[1,2,3])

In [8]:
#creating bins for the year it was built.
for x in range(len(holdout.yr_built)):
    holdout.yr_built[x]=(math.floor(holdout.yr_built[x]/10))*10

In [9]:
#used zipcode to create a column that knows if a zipcode is an expensive neighborhood or not.
high_income_zip=[98072, 98065, 98052, 98005, 8177, 98121, 98199, 98110, 98006, 98053, 98101, 98033, 98077, 98074, 98075, 98112, 98164, 98004, 98040, 98039]
holdout['high_income_zip']=0
for x in range(len(holdout.zipcode)):
    if holdout.zipcode[x] in high_income_zip:
        holdout.high_income_zip[x]=1
    else:
        holdout.high_income_zip[x]=0

In [10]:
#decided to create dummies for grade and condition to better expalin their categorical values
dummies = pd.get_dummies(holdout['grade'],prefix='grade')
dummies[list(dummies.columns)[:2]]
holdout=pd.concat([holdout, dummies[list(dummies.columns)[:2]]], axis=1, sort=False)
holdout=holdout.drop('grade',axis=1)

holdout = pd.concat([holdout, pd.get_dummies(holdout['condition'],drop_first = True,prefix = 'condition')], 1)

In [11]:
#created a binary variable that veryfies if a property has been resold or not. 
resold=holdout.id.value_counts(sort=True).to_frame()
res=list(holdout[holdout.id.isin(list(resold[resold.id>1].index))].index)
holdout['resold']=0
for x in res:
    holdout.resold[x]=1

In [12]:
#created a column that shows the age of a house since it was renovated.
conditions=[
    holdout.yr_renovated==0,
    holdout.yr_renovated!=0,
]
choises=[
    2020-holdout.yr_built,
    (2020 - holdout['yr_renovated']) 
]
holdout['age']=np.select(conditions,choises)

In [13]:
#creating polinomials to try to make a more linear relation for the features.
holdout['sqft_lot15_log'] = np.log(holdout['sqft_lot15'])
holdout['dis_from_city_log'] = np.log(holdout['dis_from_city'])
holdout['sqft_basement^2'] = holdout['sqft_basement']**2

In [14]:
#creating interactions with features that I think have relevance with eachother.
holdout['sqft_living_waterfront'] = holdout['sqft_living']*holdout['waterfront']
holdout['sqft_lot_high_income_zip'] = holdout['sqft_lot']*holdout['high_income_zip']
holdout['sqft_above_waterfront'] = holdout['sqft_above']*holdout['floors']
holdout['yr_built_sqrft_living'] = holdout['yr_built']*holdout['sqft_living']

In [15]:
#dropping unneeded columns
holdout.drop('date',axis=1,inplace=True)

In [16]:
transformed_holdout =pd.DataFrame(data=final_scaler.transform(holdout),columns=holdout.columns)

## Step 3: Predict the holdout set

In [17]:
final_answers = final_model.predict(transformed_holdout)

In [21]:
final_model.coef_

array([-7.97604898e-07, -2.66971881e+04,  4.09244298e+04,  1.19039767e+03,
        2.35188295e-01, -1.09662762e+05, -4.43917684e+05,  5.72115298e+04,
        4.64057385e+04,  6.24455451e+02,  5.67019017e+02,  7.51749731e+02,
        1.66414756e+01, -7.69070670e+02,  3.93950568e+01, -4.52913274e-02,
        3.62140553e+04,  1.35285998e+03, -2.93353313e+03,  1.36489235e+05,
       -1.84142518e+05, -2.04588410e+05,  2.97008602e+04,  2.03666117e+04,
       -1.52812061e+03, -4.86099595e+03, -1.67409416e+04, -3.79328449e+02,
        8.61625871e+03, -1.96278790e+05,  4.07290097e-02,  3.29747720e+02,
       -1.60852328e-01,  4.78435249e+01, -8.60377688e-01])

In [22]:
transformed_holdout.describe()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,year,month,dis_from_city,high_income_zip,grade_1,grade_2,condition_2,condition_3,condition_4,condition_5,resold,age,sqft_lot15_log,dis_from_city_log,sqft_basement^2,sqft_living_waterfront,sqft_lot_high_income_zip,sqft_above_waterfront,yr_built_sqrft_living
count,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000
mean,0.020840,0.015715,0.272713,0.110740,-0.058937,0.460964,-0.016908,-0.048078,-0.245769,0.177413,-0.102086,0.499054,-0.048980,-0.007746,0.064219,-0.076559,-0.006386,0.002706,-0.022447,-0.027365,-0.031701,-0.057284,-0.022676,0.287011,-0.242825,-0.103548,-0.024347,-0.484575,-0.331056,-0.064543,-0.083229,-0.020433,-0.067820,0.302681,0.126543
std,1.015928,0.964546,1.015067,1.050845,0.960666,1.087772,0.901620,0.917605,0.852138,1.079599,0.921959,1.042333,0.879980,0.950908,1.063065,1.006931,0.997677,1.002534,1.043808,0.981808,0.849439,1.079029,0.865919,0.858401,0.822908,0.816314,0.889824,1.013812,1.115788,1.080700,0.835248,0.801146,0.698596,1.042300,1.056632
min,-1.586984,-3.590337,-2.741167,-1.947261,-0.345324,-0.909850,-0.089698,-0.307685,-3.698124,-1.808046,-0.659293,-2.249073,-0.209349,-1.439999,-1.997694,-0.448916,-0.690941,-1.787207,-1.627851,-0.543081,-0.112739,-3.230354,-0.090025,-1.363724,-0.594584,-0.293003,-0.115599,-1.596436,-3.064482,-3.905138,-0.435192,-0.080011,-0.228569,-1.243060,-1.926168
25%,-0.848741,-0.396532,-0.469779,-0.684207,-0.270496,-0.909850,-0.089698,-0.307685,-0.627233,-0.652773,-0.659293,-0.207342,-0.209349,-0.843056,-0.741822,-0.325911,-0.690941,-0.824715,-0.899246,-0.543081,-0.112739,0.309564,-0.090025,0.733286,-0.594584,-0.293003,-0.115599,-1.079911,-0.829306,-0.819696,-0.435192,-0.080011,-0.228569,-0.624575,-0.665753
50%,-0.254473,-0.396532,0.503673,-0.088549,-0.218503,0.945804,-0.089698,-0.307685,-0.627233,-0.119755,-0.659293,1.153812,-0.209349,-0.246112,-0.172298,-0.252454,-0.690941,0.137777,-0.202319,-0.543081,-0.112739,0.309564,-0.090025,0.733286,-0.594584,-0.293003,-0.115599,-1.079911,-0.330123,0.099454,-0.435192,-0.080011,-0.228569,0.169940,-0.087781
75%,1.022239,0.668070,0.503673,0.704936,-0.143899,0.945804,-0.089698,-0.307685,-0.627233,0.881160,0.285106,1.153812,-0.209349,0.723922,0.689289,-0.142268,1.447302,0.779439,0.675628,-0.543081,-0.112739,0.309564,-0.090025,0.733286,-0.594584,-0.293003,-0.115599,0.297489,0.169059,0.786603,-0.164643,-0.080011,-0.228569,1.016789,0.725169
max,1.854014,7.055681,5.695417,6.454986,27.173953,3.729284,11.148536,4.852229,2.443659,7.513730,5.681831,1.494101,4.824151,2.253590,6.165478,31.045263,1.447302,1.741931,5.175483,1.841346,8.870029,0.309564,11.108037,0.733286,1.681847,3.412936,8.650626,2.363590,5.780001,2.385850,11.762073,17.049528,24.652074,8.759747,6.509822


## Step 4: Export your predictions

In [19]:
final_answers=pd.DataFrame(final_answers)
final_answers.columns=['Predictions']
final_answers

,Predictions
0,1.500525e+06
1,1.575355e+06
2,1.248981e+06
3,1.771398e+06
4,1.410909e+06
...,...
4318,1.321982e+06
4319,1.577434e+06
4320,1.835165e+06
4321,1.379726e+06


In [20]:
final_answers.to_csv('housing_preds_your_ignacio_ruiz.csv')